## 22.1 约束
&emsp;&emsp;DBMS 通过在数据库表上施加约束来实施引用完整性。大多数约束是在表定义中定义的。

>**约束（constraint）**：管理如何插入或处理数据库数据的规则。

>**注意**：有几种不同类型的约束，每个DBMS 都提供自己的支持。因此，这里给出的例子在不同的DBMS 上可能有不同的反应。在进行试验之前，请参阅具体的DBMS 文档。

### 22.1.1 主键
&emsp;&emsp;主键是一种特殊的约束，用来保证一列（或一组列）中的值是唯一的，而且永不改动。换句话说，表中的一列（或多个列）的值唯一标识表中的每一行。这方便了直接或交互地处理表中的行。没有主键，要安全地UPDATE 或DELETE 特定行而不影响其他行会非常困难。表中任意列只要满足以下条件，都可以用于主键：
- 任意两行的主键值都不相同；
- 每行都具有一个主键值（即列中不允许NULL 值）；
- 包含主键值的列从不修改或更新；
- 主键值不能重用。如果从表中删除某一行，其主键值不分配给新行。

&emsp;&emsp;一种定义主键的方法是创建它：

``` SQL
CREATE TABLE Vendors (
vend_id CHAR(10) NOT NULL PRIMARY KEY,
vend_name CHAR(50) NOT NULL,
vend_address CHAR(50) NULL,
vend_city CHAR(50) NULL,
vend_state CHAR(5) NULL,
vend_zip CHAR(10) NULL,
vend_country CHAR(50) NULL
);
```

&emsp;&emsp;另外一种则是用CONSTRAINT给列定义添加关键字PRIMARY KEY，使其成为主键：

``` SQL
ALTER TABLE Vendors
ADD CONSTRAINT PRIMARY KEY (vend_id);
```

>**说明**：SQLite 不允许使用ALTER TABLE 定义键，要求在初始的CREATE TABLE语句中定义它们。

### 22.1.2 外键
&emsp;&emsp;外键是表中的一列，其值必须列在另一表的主键中。外键是保证引用完整性的极其重要部分。

&emsp;&emsp;我们举个例子来理解外键。Orders 表将录入到系统的每个订单作为一行包含其中。顾客信息存储在Customers 表中。Orders 表中的订单通过顾客ID 与Customers 表中的特定行相关联。顾客ID 为Customers 表的主键，每个顾客都有唯一的ID。订单号为Orders 表的主键，每个订单都有唯一的订单号。Orders 表中顾客ID 列的值不一定是唯一的。如果某个顾客有多个订单，则有多个行具有相同的顾客ID（虽然每个订单都有不同的订单号）。同时，Orders 表中顾客ID 列的合法值为Customers 表中顾客的ID。这就是外键的作用。在这个例子中，在Orders 的顾客ID 列上定义了一个外键，因此该列只能接受Customers 表的主键值。

``` SQL
-- 定义外键的方法之一
CREATE TABLE Orders
(
order_num INTEGER NOT NULL PRIMARY KEY,
order_date DATETIME NOT NULL,
cust_id CHAR(10) NOT NULL REFERENCES Customers(cust_id)
);
```

``` SQL
-- 定义外键的方法之二
ALTER TABLE Orders
ADD CONSTRAINT
FOREIGN KEY (cust_id) REFERENCES Customers (cust_id);
```

>**提示**：在定义外键后，DBMS 不允许删除在另一个表中具有关联行的行。例如，不能删除关联订单的顾客。删除该顾客的唯一方法是首先删除相关的订单（这表示还要删除相关的订单项）。由于需要一系列的删除，因而利用外键可以防止意外删除数据。有的DBMS 支持称为级联删除（cascading delete）的特性。如果启用，该特性在从一个表中删除行时删除所有相关的数据。例如，如果启用级联删除并且从Customers 表中删除某个顾客，则任何关联的订单行也会被自动删除。

### 22.1.3 唯一约束
&emsp;&emsp;唯一约束用来保证一列（或一组列）中的数据是唯一的。它们类似于主键，但存在以下重要区别：
- 表可包含多个唯一约束，但每个表只允许一个主键；
- 唯一约束列可包含NULL 值；
- 唯一约束列可修改或更新；
- 唯一约束列的值可重复使用；
- 与主键不一样，唯一约束不能用来定义外键。

&emsp;&emsp;唯一约束的语法类似于其他约束的语法。唯一约束既可以用UNIQUE 关键字在表定义中定义，也可以用单独的CONSTRAINT 定义。

### 22.1.4 检查约束
&emsp;&emsp;检查约束用来保证一列（或一组列）中的数据满足一组指定的条件。检查约束的常见用途有以下几点：
- 检查最小或最大值。例如，防止0 个物品的订单（即使0 是合法的数）；
- 指定范围。例如，保证发货日期大于等于今天的日期，但不超过今天起一年后的日期’
- 只允许特定的值。例如，在性别字段中只允许M 或F。

``` SQL
-- 施加检查约束
CREATE TABLE OrderItems (
order_num INTEGER NOT NULL,
order_item INTEGER NOT NULL,
prod_id CHAR(10) NOT NULL,
quantity INTEGER NOT NULL CHECK (quantity > 0),
item_price MONEY NOT NULL
);
```

``` SQL
-- ALTER TABLE施加检查约束
ADD CONSTRAINT CHECK (gender LIKE '[MF]')
);
```

>**提示**：有的DBMS 允许用户定义自己的数据类型。它们是定义检查约束（或其他约束）的基本简单数据类型。例如，你可以定义自己的名为gender的数据类型，它是单字符的文本数据类型，带限制其值为M 或F（对于未知值或许还允许NULL）的检查约束。然后，可以将此数据类型用于表的定义。定制数据类型的优点是只需施加约束一次（在数据类型定义中），而每当使用该数据类型时，都会自动应用这些约束。请查阅相应的DBMS 文档，看它是否支持自定义数据类型。

## 22.2 索引
&emsp;&emsp;索引用来排序数据以加快搜索和排序操作的速度。可以在一个或多个列上定义索引，使DBMS 保存其内容的一个排过序的列表。在定义了索引后，DBMS 以使用书的索引类似的方法使用它。DBMS 搜索排过序的索引，找出匹配的位置，然后检索这些行。

&emsp;&emsp;在开始创建索引前，应该记住以下内容：
- 索引改善检索操作的性能，但降低了数据插入、修改和删除的性能，在执行这些操作时，DBMS 必须动态地更新索引；
- 索引数据可能要占用大量的存储空间；
- 并非所有数据都适合做索引。取值不多的数据（如州）不如具有更多可能值的数据（如姓或名），能通过索引得到那么多的好处；
- 索引用于数据过滤和数据排序。如果你经常以某种特定的顺序排序数据，则该数据可能适合做索引；
- 可以在索引中定义多个列（例如，州加上城市）。这样的索引仅在以州加城市的顺序排序时有用。如果想按城市排序，则这种索引没有用处。

&emsp;&emsp;索引用CREATE INDEX 语句创建（不同DBMS创建索引的语句变化很大）。

``` SQL
CREATE INDEX prod_name_ind
ON Products (prod_name);
```

>**提示**：索引的效率随表数据的增加或改变而变化。许多数据库管理员发现，过去创建的某个理想的索引经过几个月的数据处理后可能变得不再理想了。最好定期检查索引，并根据需要对索引进行调整。

## 22.3 触发器
&emsp;&emsp;触发器是特殊的存储过程，它在特定的数据库活动发生时自动执行。触发器可以与特定表上的INSERT、UPDATE 和DELETE 操作（或组合）相关联。与存储过程不一样（存储过程只是简单的存储SQL 语句），触发器与单个的表相关联。

&emsp;&emsp;触发器内的代码具有以下数据的访问权：
- INSERT 操作中的所有新数据；
- UPDATE 操作中的所有新数据和旧数据；
- DELETE 操作中删除的数据。

&emsp;&emsp;下面是触发器的一些常见用途：
- 保证数据一致。例如，在INSERT 或UPDATE 操作中将所有州名转换为大写；
- 基于某个表的变动在其他表上执行活动。例如，每当更新或删除一行时将审计跟踪记录写入某个日志表；
- 进行额外的验证并根据需要回退数据。例如，保证某个顾客的可用资金不超限定，如果已经超出，则阻塞插入；
- 计算计算列的值或更新时间戳。

&emsp;&emsp;不同DBMS 的触发器创建语法差异很大，更详细的信息请参阅相应的文档。

``` SQL
-- SQL Server版本
CREATE TRIGGER customer_state
ON Customers
FOR INSERT, UPDATE
AS
UPDATE Customers
SET cust_state = Upper(cust_state)
WHERE Customers.cust_id = inserted.cust_id;
```

``` SQL
-- Oracle 和PostgreSQL 的版本
CREATE TRIGGER customer_state
AFTER INSERT OR UPDATE
FOR EACH ROW
BEGIN
UPDATE Customers
SET cust_state = Upper(cust_state)
WHERE Customers.cust_id = :OLD.cust_id
END;
```

>**提示**：一般来说，约束的处理比触发器快，因此在可能的时候，应该尽量使用约束。

## 22.4 数据库安全
&emsp;&emsp;一般说来，需要保护的操作有：
- 对数据库管理功能（创建表、更改或删除已存在的表等）的访问；
- 对特定数据库或表的访问；
-访问的类型（只读、对特定列的访问等）；
- 仅通过视图或存储过程对表进行访问；
- 创建多层次的安全措施，从而允许多种基于登录的访问和控制；
- 限制管理用户账号的能力。

&emsp;&emsp;安全性使用SQL 的GRANT 和REVOKE 语句来管理，不过，大多数DBMS提供了交互式的管理实用程序，这些实用程序在内部使用GRANT 和REVOKE 语句。

## 22.5 小结
&emsp;&emsp;本章学习了如何使用SQL 的一些高级特性。约束是实施引用完整性的重要部分，索引可改善数据检索的性能，触发器可以用来执行运行前后的处理，安全选项可用来管理数据访问。不同的DBMS 可能会以不同的形式提供这些特性，更详细的信息请参阅具体的DBMS 文档。